![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Installs

In [ ]:
# install PySpark
# %pip install -q pyspark==3.3.1 spark-nlp==5.1.0

In [ ]:
# %pip install spacy[transformers]

# Initializing Spark

In [ ]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *

from pyspark.ml import Pipeline

print("Spark NLP version", sparknlp.version())

spark

In [ ]:
!pip show spacy

Name: spacy
Version: 3.6.1
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /databricks/python3/lib/python3.8/site-packages
Requires: tqdm, spacy-loggers, wasabi, preshed, requests, typer, pydantic, spacy-legacy, numpy, setuptools, catalogue, murmurhash, srsly, smart-open, pathy, jinja2, packaging, thinc, cymem, langcodes
Required-by: spacy-transformers, en-core-web-trf, en-core-web-sm

In [ ]:
spark.conf.get("spark.sql.execution.arrow.pyspark.enabled")

Out[4]: 'true'

In [ ]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.get("spark.sql.execution.arrow.pyspark.enabled")

Out[5]: 'false'

# Get Data

In [ ]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/open-source-nlp/data/news_category_train.csv

In [ ]:
dbutils.fs.cp("file:/databricks/driver/news_category_train.csv", "dbfs:/") 

Out[7]: True

In [ ]:
spark_df = spark.read.csv('/news_category_train.csv').toDF("category",'text')
spark_df.write.mode("overwrite").parquet("/Users/halil@johnsnowlabs.com/temp/news")
spark_df= spark.read.parquet("/Users/halil@johnsnowlabs.com/temp/news").repartition(10000)
spark_df.show(5,False)

+--------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
category|text |
+--------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
Sports |This past Sunday night (or Sunday afternoon Eastern Standard Time), my roommates and I hung our heads in defeat as Colin Montgomerie put away the Americans in the Ryder Cup. |
Sci/Tech| A swarm of earthquakes #151; one a magnitude 5.5 #151; jolted a remote, sparsely populated area along the eastern Sierra Nevada on Saturday, authorities said. |
World | U.S. officials have arrested two men whom they accuse of funneling money to the Palestinian militant group Hamas for the past 15 years, the Justice Department said on Friday. |
World |By John Deane, Chief Political Correspondent, PA News, in Darfur. Foreign Secretary Jack Straw arrived in the violence-plagued Darfur region of Sudan today to experience at first hand the plight of refugees driven from their homes by marauding militiamen.|
Sci/Tech|Servers for small businesses are one of the fastest-growing areas of the IT market, according to market researcher IDC. Much of this is down to the Small Business Server (SBS) software |
+--------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 5 rows

In [ ]:
spark_df.count()

Out[9]: 120001

# Spark NLP

In [ ]:
from sparknlp.base import *
from sparknlp.annotator import *

NER extraction

In [ ]:
document_assembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")\

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

glove_embeddings = WordEmbeddingsModel.pretrained('glove_100d').\
    setInputCols(["document", 'token']).\
    setOutputCol("embeddings")

public_ner = NerDLModel.pretrained("ner_dl", 'en') \
    .setInputCols(["document", "token", "embeddings"]) \
    .setOutputCol("ner")

pipeline = Pipeline(stages=[document_assembler,
                              tokenizer,
                              glove_embeddings,
                              public_ner
                           ])

fitted_pipeline = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))


glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ][OK!]
ner_dl download started this may take some time.
Approximate size to download 13.6 MB
[ | ][OK!]

In [ ]:
%%time
spark_result = fitted_pipeline.transform(spark_df)

spark_result.select('ner').collect() 

CPU times: user 39.4 s, sys: 16.5 s, total: 55.9 s
Wall time: 3min 35s
Out[14]: [Row(ner=[Row(annotatorType='named_entity', begin=0, end=4, result='B-LOC', metadata={'sentence': '0', 'word': 'MIAMI'}, embeddings=[]), Row(annotatorType='named_entity', begin=5, end=5, result='O', metadata={'sentence': '0', 'word': ','}, embeddings=[]), Row(annotatorType='named_entity', begin=7, end=9, result='O', metadata={'sentence': '0', 'word': 'Oct'}, embeddings=[]), Row(annotatorType='named_entity', begin=11, end=12, result='O', metadata={'sentence': '0', 'word': '21'}, embeddings=[]), Row(annotatorType='named_entity', begin=14, end=14, result='O', metadata={'sentence': '0', 'word': '('}, embeddings=[]), Row(annotatorType='named_entity', begin=15, end=17, result='O', metadata={'sentence': '0', 'word': 'AFP'}, embeddings=[]), Row(annotatorType='named_entity', begin=18, end=18, result='O', metadata={'sentence': '0', 'word': ')'}, embeddings=[]), Row(annotatorType='named_entity', begin=20, end=20, result='O', metadata={'sentence': '0', 'word': '-'}, embeddings=[]), Row(annotatorType='named_entity', begin=22, end=26, result='O', metadata={'sentence': '0', 'word': 'Known'}, embeddings=[]), Row(annotatorType='named_entity', begin=28, end=29, result='O', metadata={'sentence': '0', 'word': 'as'}, embeddings=[]), Row(annotatorType='named_entity', begin=31, end=33, result='O', metadata={'sentence': '0', 'word': 'the'}, embeddings=[]), Row(annotatorType='named_entity', begin=35, end=41, result='O', metadata={'sentence': '0', 'word': 'capital'}, embeddings=[]), Row(annotatorType='named_entity', begin=43, end=44, result='O', metadata={'sentence': '0', 'word': 'of'}, embeddings=[]), Row(annotatorType='named_entity', begin=46, end=50, result='B-MISC', metadata={'sentence': '0', 'word': 'Cuban'}, embeddings=[]), Row(annotatorType='named_entity', begin=52, end=57, result='O', metadata={'sentence': '0', 'word': 'exiles'}, embeddings=[]), Row(annotatorType='named_entity', begin=58, end=58, result='O', metadata={'sentence': '0', 'word': ','}, embeddings=[]), Row(annotatorType='named_entity', begin=60, end=64, result='B-LOC', metadata={'sentence': '0', 'word': 'Miami'}, embeddings=[]), Row(annotatorType='named_entity', begin=66, end=68, result='O', metadata={'sentence': '0', 'word': 'was'}, embeddings=[]), Row(annotatorType='named_entity', begin=70, end=74, result='O', metadata={'sentence': '0', 'word': 'abuzz'}, embeddings=[]), Row(annotatorType='named_entity', begin=76, end=83, result='O', metadata={'sentence': '0', 'word': 'Thursday'}, embeddings=[]), Row(annotatorType='named_entity', begin=85, end=88, result='O', metadata={'sentence': '0', 'word': 'with'}, embeddings=[]), Row(annotatorType='named_entity', begin=90, end=93, result='O', metadata={'sentence': '0', 'word': 'news'}, embeddings=[]), Row(annotatorType='named_entity', begin=95, end=96, result='O', metadata={'sentence': '0', 'word': 'of'}, embeddings=[]), Row(annotatorType='named_entity', begin=98, end=106, result='O', metadata={'sentence': '0', 'word': 'President'}, embeddings=[]), Row(annotatorType='named_entity', begin=108, end=112, result='B-PER', metadata={'sentence': '0', 'word': 'Fidel'}, embeddings=[]), Row(annotatorType='named_entity', begin=114, end=119, result='I-PER', metadata={'sentence': '0', 'word': 'Castro'}, embeddings=[]), Row(annotatorType='named_entity', begin=121, end=125, result='O', metadata={'sentence': '0', 'word': '#39;s'}, embeddings=[]), Row(annotatorType='named_entity', begin=127, end=134, result='O', metadata={'sentence': '0', 'word': 'accident'}, embeddings=[]), Row(annotatorType='named_entity', begin=135, end=135, result='O', metadata={'sentence': '0', 'word': ','}, embeddings=[]), Row(annotatorType='named_entity', begin=137, end=140, result='O', metadata={'sentence': '0', 'word': 'with'}, embeddings=[]), Row(annotatorType='named_entity', begin=142, end=145, result='O', metadata={'sentence': '0', 'word': 'many'}, embeddings=[]), Row(annotatorType='named_entity', begin

Roberta sentence embeddings

In [ ]:
document_assembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")\

embeddings = RoBertaSentenceEmbeddings.pretrained("sent_roberta_base", "en") \
      .setInputCols("document") \
      .setOutputCol("embeddings")

pipeline= Pipeline(stages=[document_assembler,
                            embeddings
                           ])

fitted_pipeline= pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

sent_roberta_base download started this may take some time.
Approximate size to download 284.8 MB
[ | ][OK!]

In [ ]:
%%time

result= fitted_pipeline.transform(spark_df)
result.select("embeddings.embeddings").collect() 

CPU times: user 16.5 s, sys: 4.59 s, total: 21 s
Wall time: 22min 16s
Out[10]: [Row(embeddings=[[-0.011482356116175652, -0.19854967296123505, -0.22599659860134125, -0.08121123164892197, 0.14474287629127502, 0.19604481756687164, 0.24963760375976562, -0.06219545006752014, -0.09065160155296326, -0.16833780705928802, 0.22114494442939758, -0.0012329714372754097, -0.09805963188409805, 0.08891163021326065, -0.109325110912323, 0.47480636835098267, 0.21826697885990143, -0.4562963545322418, 0.04325294494628906, -0.03163160756230354, -0.22714857757091522, 0.061855144798755646, 0.46584758162498474, 0.3259323537349701, 0.11403614282608032, 0.07397361099720001, -0.14958882331848145, -0.015807345509529114, 0.17977726459503174, 0.24030731618404388, 0.2917097210884094, 0.04745013639330864, 0.08286707103252411, 0.24748218059539795, -0.23676343262195587, 0.04940634220838547, -0.30726519227027893, 0.017717288807034492, 0.2503613531589508, -0.20105144381523132, -0.07013659179210663, 0.17356058955192566, 0.20971333980560303, -0.15701153874397278, -0.1072697713971138, 0.41679325699806213, 0.24923130869865417, 0.022938545793294907, -0.12161150574684143, -0.08386552333831787, -0.3643246591091156, 0.3432160019874573, 0.29178333282470703, 0.18252769112586975, -0.0025009959936141968, 0.036416564136743546, -0.15584918856620789, 0.2564684748649597, -0.08708812296390533, -0.11099347472190857, -0.10982109606266022, -0.2153463512659073, 3.975117579102516e-06, -0.056173063814640045, 0.015296602621674538, -0.13171082735061646, 0.07058795541524887, -0.1473367065191269, -0.1289597898721695, 0.07195441424846649, -0.0714956596493721, 0.15160039067268372, 0.17342251539230347, -0.30083566904067993, -0.26048386096954346, 0.04445672407746315, -0.5935322046279907, -0.11065396666526794, 0.29427289962768555, 0.4142172634601593, -0.11021408438682556, 0.17112530767917633, 0.07070452719926834, 0.24892199039459229, -0.014725763350725174, -0.06518939137458801, -0.04893629252910614, -0.11282088607549667, 0.1800653636455536, 0.2673394978046417, -0.17937736213207245, -0.3862754702568054, 0.0604107566177845, 0.03345911577343941, -0.09744397550821304, 0.031104564666748047, 0.003002037527039647, -0.08159651607275009, -0.16133293509483337, -0.20557041466236115, 0.05700211226940155, -0.2827898859977722, -0.16344821453094482, 0.25773563981056213, -0.012171322479844093, -0.1805790662765503, -0.023465201258659363, 0.31301093101501465, 0.06964049488306046, -0.12741869688034058, -0.17893941700458527, 0.41420647501945496, 0.304250568151474, 0.014318086206912994, 0.019374025985598564, 0.16543620824813843, 0.11197630316019058, -0.28005450963974, 0.41709011793136597, -0.27678173780441284, 0.008869998157024384, -0.09916847944259644, 0.12345714122056961, 0.15085427463054657, -0.2070542722940445, 0.26972445845603943, 0.14354868233203888, 0.2248515635728836, 0.18255095183849335, 0.09761152416467667, -0.052858442068099976, 0.09697911888360977, -0.12590204179286957, 0.17293305695056915, 0.23900313675403595, 0.1044442430138588, -0.005584129597991705, -0.3315146863460541, -0.21684810519218445, 0.25984710454940796, 0.32422512769699097, 0.16883984208106995, -0.03615548834204674, 0.17544524371623993, 0.1020113155245781, 0.23570173978805542, 0.12625624239444733, -0.391092985868454, 0.04903579503297806, 0.3567868173122406, 0.11720843613147736, 0.1731446236371994, -0.0810665562748909, -0.26809632778167725, -0.22422412037849426, -0.08992619812488556, 0.025666125118732452, -0.3017897605895996, -0.13195690512657166, 0.35391005873680115, 0.008953727781772614, 0.0004345481575001031, -0.13859666883945465, -0.2043808102607727, -0.0491613931953907, -0.10549143701791763, 0.04379435256123543, 0.0805233046412468, -0.07647501677274704, -0.40334776043891907, -0.09208035469055176, -0.5487066507339478, -0.12331181019544601, 0.18022005259990692, -0.29391124844551086, 0.23556530475616455, -0.2869839668273926, 0.1062140017747879, 0.3852437436580658, 0.027243487536907196, -0.007322994526475668, -0.1565389782190323, -0.0282410

# Define UDF Functions

In [ ]:
import spacy
from pyspark.sql.functions import pandas_udf
import pandas as pd
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import Row

from pyspark.sql.types import ArrayType, FloatType, StringType

In [ ]:
! python -m spacy download en_core_web_sm

Collecting en-core-web-sm==3.6.0
 Downloading https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.6.0/en_core_web_sm-3.6.0-py3-none-any.whl (12.8 MB)
 | 10 kB 12.5 MB/s eta 0:00:02 | | 20 kB 18.2 MB/s eta 0:00:01 | | 30 kB 22.5 MB/s eta 0:00:01 | | 40 kB 13.3 MB/s eta 0:00:01 |▏ | 51 kB 15.2 MB/s eta 0:00:01 |▏ | 61 kB 16.7 MB/s eta 0:00:01 |▏ | 71 kB 14.0 MB/s eta 0:00:01 |▏ | 81 kB 15.5 MB/s eta 0:00:01 |▎ | 92 kB 16.9 MB/s eta 0:00:01 |▎ | 102 kB 15.8 MB/s eta 0:00:01 |▎ | 112 kB 15.8 MB/s eta 0:00:01 |▎ | 122 kB 15.8 MB/s eta 0:00:01 |▎ | 133 kB 15.8 MB/s eta 0:00:01 |▍ | 143 kB 15.8 MB/s eta 0:00:01 |▍ | 153 kB 15.8 MB/s eta 0:00:01 |▍ | 163 kB 15.8 MB/s eta 0:00:01 |▍ | 174 kB 15.8 MB/s eta 0:00:01 |▌ | 184 kB 15.8 MB/s eta 0:00:01 |▌ | 194 kB 15.8 MB/s eta 0:00:01 |▌ | 204 kB 15.8 MB/s eta 0:00:01 |▌ | 215 kB 15.8 MB/s eta 0:00:01 |▋ | 225 kB 15.8 MB/s eta 0:00:01 |▋ | 235 kB 15.8 MB/s eta 0:00:01 |▋ | 245 kB 15.8 MB/s eta 0:00:01 |▋ | 256 kB 15.8 MB/s eta 0:00:01 |▋ | 266 kB 15.8 MB/s eta 0:00:01 |▊ | 276 kB 15.8 MB/s eta 0:00:01 |▊ | 286 kB 15.8 MB/s eta 0:00:01 |▊ | 296 kB 15.8 MB/s eta 0:00:01 |▊ | 307 kB 15.8 MB/s eta 0:00:01 |▉ | 317 kB 15.8 MB/s eta 0:00:01 |▉ | 327 kB 15.8 MB/s eta 0:00:01 |▉ | 337 kB 15.8 MB/s eta 0:00:01 |▉ | 348 kB 15.8 MB/s eta 0:00:01 |█ | 358 kB 15.8 MB/s eta 0:00:01 |█ | 368 kB 15.8 MB/s eta 0:00:01 |█ | 378 kB 15.8 MB/s eta 0:00:01 |█ | 389 kB 15.8 MB/s eta 0:00:01 |█ | 399 kB 15.8 MB/s eta 0:00:01 |█ | 409 kB 15.8 MB/s eta 0:00:01 |█ | 419 kB 15.8 MB/s eta 0:00:01 |█ | 430 kB 15.8 MB/s eta 0:00:01 |█ | 440 kB 15.8 MB/s eta 0:00:01 |█▏ | 450 kB 15.8 MB/s eta 0:00:01 |█▏ | 460 kB 15.8 MB/s eta 0:00:01 |█▏ | 471 kB 15.8 MB/s eta 0:00:01 |█▏ | 481 kB 15.8 MB/s eta 0:00:01 |█▎ | 491 kB 15.8 MB/s eta 0:00:01 |█▎ | 501 kB 15.8 MB/s eta 0:00:01 |█▎ | 512 kB 15.8 MB/s eta 0:00:01 |█▎ | 522 kB 15.8 MB/s eta 0:00:01 |█▎ | 532 kB 15.8 MB/s eta 0:00:01 |█▍ | 542 kB 15.8 MB/s eta 0:00:01 |█▍ | 552 kB 15.8 MB/s eta 0:00:01 |█▍ | 563 kB 15.8 MB/s eta 0:00:01 |█▍ | 573 kB 15.8 MB/s eta 0:00:01 |█▌ | 583 kB 15.8 MB/s eta 0:00:01 |█▌ | 593 kB 15.8 MB/s eta 0:00:01 |█▌ | 604 kB 15.8 MB/s eta 0:00:01 |█▌ | 614 kB 15.8 MB/s eta 0:00:01 |█▋ | 624 kB 15.8 MB/s eta 0:00:01 |█▋ | 634 kB 15.8 MB/s eta 0:00:01 |█▋ | 645 kB 15.8 MB/s eta 0:00:01 |█▋ | 655 kB 15.8 MB/s eta 0:00:01 |█▋ | 665 kB 15.8 MB/s eta 0:00:01 |█▊ | 675 kB 15.8 MB/s eta 0:00:01 |█▊ | 686 kB 15.8 MB/s eta 0:00:01 |█▊ | 696 kB 15.8 MB/s eta 0:00:01 |█▊ | 706 kB 15.8 MB/s eta 0:00:01 |█▉ | 716 kB 15.8 MB/s eta 0:00:01 |█▉ | 727 kB 15.8 MB/s eta 0:00:01 |█▉ | 737 kB 15.8 MB/s eta 0:00:01 |█▉ | 747 kB 15.8 MB/s eta 0:00:01 |██ | 757 kB 15.8 MB/s eta 0:00:01 |██ | 768 kB 15.8 MB/s eta 0:00:01 |██ | 778 kB 15.8 MB/s eta 0:00:01 |██ | 788 kB 15.8 MB/s eta 0:00:01 |██ | 798 kB 15.8 MB/s eta 0:00:01 |██ | 808 kB 15.8 MB/s eta 0:00:01 |██ | 819 kB 15.8 MB/s eta 0:00:01 |██ | 829 kB 15.8 MB/s eta 0:00:01 |██ | 839 kB 15.8 MB/s eta 0:00:01 |██▏ | 849 kB 15.8 MB/s eta 0:00:01 |██▏ | 860 kB 15.8 MB/s eta 0:00:01 |██▏ | 870 kB 15.8 MB/s eta 0:00:01 |██▏ | 880 kB 15.8 MB/s eta 0:00:01 |██▎ | 890 kB 15.8 MB/s eta 0:00:01 |██▎ | 901 kB 15.8 MB/s eta 0:00:01 |██▎ | 911 kB 15.8 MB/s eta 0:00:01 |██▎ | 921 kB 15.8 MB/s eta 0:00:01 |██▎ | 931 kB 15.8 MB/s eta 0:00:01 |██▍ | 942 kB 15.8 MB/s eta 0:00:01 |██▍ | 952 kB 15.8 MB/s eta 0:00:01 |██▍ | 962 kB 15.8 MB/s eta 0:00:01 |██▍ | 972 kB 15.8 MB/s eta 0:00:01 |██▌ | 983 kB 15.8 MB/s eta 0:00:01 |██▌ | 993 kB 15.8 MB/s eta 0:00:01 |██▌ | 1.0 MB 15.8 MB/s eta 0:00:01 |██▌ | 1.0 MB 15.8 MB/s eta 0:00:01 |██▋ | 1.0 MB 15.8 MB/s eta 0:00:01 |██▋ | 1.0 MB 15.8 MB/s eta 0:00:01 |██▋ | 1.0 MB 15.8 MB/s eta 0:00:01 |██▋ | 1.1 MB 15.8 MB/s eta 0:00:01 |██▋ | 1.1 MB 15.8 MB/s eta 0:00:01 |██▊ | 1.1 MB 15.8 MB/s eta 0:00:01 |██▊ | 1.1 MB 15.8 MB/s eta 0:00:01 |██▊ | 1.1 MB 15.8 MB/s eta 0:00:01 |██

In [ ]:
# Initialize SpaCy
nlp_token = spacy.load("en_core_web_sm", exclude=["ner","tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

# Define a UDF that takes a column of text data as input 
@pandas_udf(ArrayType(StringType()))
def tokenize_with_spacy(text_series):
    tokenized_text = []
    for text in text_series:
        doc = nlp_token(text)
        tokens = [token.text for token in doc]
        tokenized_text.append(tokens)

    # Convert the tokenized_text list to a Pandas Series
    result_series = pd.Series(tokenized_text)

    return result_series

In [ ]:
nlp_ner = spacy.load("en_core_web_sm", exclude=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

# Define a UDF to  perform NER
@pandas_udf(ArrayType(StringType()))
def ner_with_spacy(text_series):
    entities_list = []
    for text in text_series:
        doc = nlp_ner(text)
        entities = [f"{ent.text}:::{ent.label_}" for ent in doc.ents]
        entities_list.append(entities)
    return pd.Series(entities_list)

In [ ]:
!python -m spacy download en_core_web_trf

Collecting en-core-web-trf==3.6.1
 Downloading https://github.com/explosion/spacy-models/releases/download/en_core_web_trf-3.6.1/en_core_web_trf-3.6.1-py3-none-any.whl (460.3 MB)
 | 10 kB 20.3 MB/s eta 0:00:23 | | 20 kB 26.5 MB/s eta 0:00:18 | | 30 kB 31.2 MB/s eta 0:00:15 | | 40 kB 9.3 MB/s eta 0:00:50 | | 51 kB 10.9 MB/s eta 0:00:43 | | 61 kB 12.8 MB/s eta 0:00:37 | | 71 kB 12.8 MB/s eta 0:00:36 | | 81 kB 14.3 MB/s eta 0:00:33 | | 92 kB 15.6 MB/s eta 0:00:30 | | 102 kB 13.9 MB/s eta 0:00:33 | | 112 kB 13.9 MB/s eta 0:00:33 | | 122 kB 13.9 MB/s eta 0:00:33 | | 133 kB 13.9 MB/s eta 0:00:33 | | 143 kB 13.9 MB/s eta 0:00:33 | | 153 kB 13.9 MB/s eta 0:00:33 | | 163 kB 13.9 MB/s eta 0:00:33 | | 174 kB 13.9 MB/s eta 0:00:33 | | 184 kB 13.9 MB/s eta 0:00:33 | | 194 kB 13.9 MB/s eta 0:00:33 | | 204 kB 13.9 MB/s eta 0:00:33 | | 215 kB 13.9 MB/s eta 0:00:33 | | 225 kB 13.9 MB/s eta 0:00:33 | | 235 kB 13.9 MB/s eta 0:00:33 | | 245 kB 13.9 MB/s eta 0:00:33 | | 256 kB 13.9 MB/s eta 0:00:33 | | 266 kB 13.9 MB/s eta 0:00:33 | | 276 kB 13.9 MB/s eta 0:00:33 | | 286 kB 13.9 MB/s eta 0:00:33 | | 296 kB 13.9 MB/s eta 0:00:33 | | 307 kB 13.9 MB/s eta 0:00:33 | | 317 kB 13.9 MB/s eta 0:00:33 | | 327 kB 13.9 MB/s eta 0:00:33 | | 337 kB 13.9 MB/s eta 0:00:33 | | 348 kB 13.9 MB/s eta 0:00:33 | | 358 kB 13.9 MB/s eta 0:00:33 | | 368 kB 13.9 MB/s eta 0:00:33 | | 378 kB 13.9 MB/s eta 0:00:33 | | 389 kB 13.9 MB/s eta 0:00:33 | | 399 kB 13.9 MB/s eta 0:00:33 | | 409 kB 13.9 MB/s eta 0:00:33 | | 419 kB 13.9 MB/s eta 0:00:33 | | 430 kB 13.9 MB/s eta 0:00:33 | | 440 kB 13.9 MB/s eta 0:00:33 | | 450 kB 13.9 MB/s eta 0:00:33 | | 460 kB 13.9 MB/s eta 0:00:33 | | 471 kB 13.9 MB/s eta 0:00:33 | | 481 kB 13.9 MB/s eta 0:00:33 | | 491 kB 13.9 MB/s eta 0:00:33 | | 501 kB 13.9 MB/s eta 0:00:33 | | 512 kB 13.9 MB/s eta 0:00:33 | | 522 kB 13.9 MB/s eta 0:00:33 | | 532 kB 13.9 MB/s eta 0:00:33 | | 542 kB 13.9 MB/s eta 0:00:33 | | 552 kB 13.9 MB/s eta 0:00:33 | | 563 kB 13.9 MB/s eta 0:00:33 | | 573 kB 13.9 MB/s eta 0:00:33 | | 583 kB 13.9 MB/s eta 0:00:33 | | 593 kB 13.9 MB/s eta 0:00:33 | | 604 kB 13.9 MB/s eta 0:00:33 | | 614 kB 13.9 MB/s eta 0:00:33 | | 624 kB 13.9 MB/s eta 0:00:33 | | 634 kB 13.9 MB/s eta 0:00:33 | | 645 kB 13.9 MB/s eta 0:00:33 | | 655 kB 13.9 MB/s eta 0:00:33 | | 665 kB 13.9 MB/s eta 0:00:33 | | 675 kB 13.9 MB/s eta 0:00:33 | | 686 kB 13.9 MB/s eta 0:00:33 | | 696 kB 13.9 MB/s eta 0:00:33 | | 706 kB 13.9 MB/s eta 0:00:33 | | 716 kB 13.9 MB/s eta 0:00:33 | | 727 kB 13.9 MB/s eta 0:00:33 | | 737 kB 13.9 MB/s eta 0:00:33 | | 747 kB 13.9 MB/s eta 0:00:33 | | 757 kB 13.9 MB/s eta 0:00:33 | | 768 kB 13.9 MB/s eta 0:00:33 | | 778 kB 13.9 MB/s eta 0:00:33 | | 788 kB 13.9 MB/s eta 0:00:33 | | 798 kB 13.9 MB/s eta 0:00:33 | | 808 kB 13.9 MB/s eta 0:00:33 | | 819 kB 13.9 MB/s eta 0:00:33 | | 829 kB 13.9 MB/s eta 0:00:33 | | 839 kB 13.9 MB/s eta 0:00:33 | | 849 kB 13.9 MB/s eta 0:00:33 | | 860 kB 13.9 MB/s eta 0:00:33 | | 870 kB 13.9 MB/s eta 0:00:33 | | 880 kB 13.9 MB/s eta 0:00:33 | | 890 kB 13.9 MB/s eta 0:00:33 | | 901 kB 13.9 MB/s eta 0:00:33 | | 911 kB 13.9 MB/s eta 0:00:33 | | 921 kB 13.9 MB/s eta 0:00:33 | | 931 kB 13.9 MB/s eta 0:00:33 | | 942 kB 13.9 MB/s eta 0:00:33 | | 952 kB 13.9 MB/s eta 0:00:33 | | 962 kB 13.9 MB/s eta 0:00:33 | | 972 kB 13.9 MB/s eta 0:00:33 | | 983 kB 13.9 MB/s eta 0:00:33 | | 993 kB 13.9 MB/s eta 0:00:33 | | 1.0 MB 13.9 MB/s eta 0:00:33 | | 1.0 MB 13.9 MB/s eta 0:00:33 | | 1.0 MB 13.9 MB/s eta 0:00:33 | | 1.0 MB 13.9 MB/s eta 0:00:33 | | 1.0 MB 13.9 MB/s eta 0:00:33 | | 1.1 MB 13.9 MB/s eta 0:00:33 | | 1.1 MB 13.9 MB/s eta 0:00:33 | | 1.1 MB 13.9 MB/s eta 0:00:33 | | 1.1 MB 13.9 MB/s eta 0:00:33 | | 1.1 MB 13.9 MB/s eta 0:00:33 | | 1.1 MB 13.9 MB/s eta 0:00:33 | | 1.1 MB 13.9 MB/s eta 0:00:33 | | 1.1 MB 13.9 MB/s eta 0:00:33 | | 1.1 MB 13.9 MB/s eta 0:00:33 | | 1.1 MB 13.9 MB/s eta 0:00:33 | | 1.2 MB 13.9 MB/s e

In [ ]:
nlp_embeddings = spacy.load("en_core_web_trf")

# Define a UDF to get sentence embeddings 
@pandas_udf(ArrayType(FloatType()))
def embeddings_with_spacy(text_series):
    embeddings_list = []
    for text in text_series:
        doc = nlp_embeddings(text)
        embeddings = doc._.trf_data.tensors[-1][0]
        embeddings_list.append(embeddings)
    return pd.Series(embeddings_list)

# Spacy with Arrow Enabled

clear all variables, and rerun initialize spark, get data and define udf function sections 

In [ ]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [ ]:
spark.conf.get("spark.sql.execution.arrow.pyspark.enabled")

Out[16]: 'true'

NER extraction

In [ ]:
%%time

# Apply the UDF to your Spark DataFrame
result = spark_df.withColumn("named_entities", ner_with_spacy(spark_df["text"]))


result.collect() 

CPU times: user 4.19 s, sys: 1.03 s, total: 5.22 s
Wall time: 4min 49s
Out[18]: [Row(category='Sci/Tech', text='Servers for small businesses are one of the fastest-growing areas of the IT market, according to market researcher IDC. Much of this is down to the Small Business Server (SBS) software ', named_entities=['IDC:::ORG', 'the Small Business Server:::ORG']),
 Row(category='World', text=' The U.N. Security Council was set Friday to adopt a Russian drafted resolution that seeks to expand the prosecution and extradition of terrorist groups and individuals, such as Chechen separatist leaders.', named_entities=['The U.N. Security Council:::ORG', 'Friday:::DATE', 'Russian:::NORP', 'Chechen:::NORP']),
 Row(category='Sports', text='This past Sunday night (or Sunday afternoon Eastern Standard Time), my roommates and I hung our heads in defeat as Colin Montgomerie put away the Americans in the Ryder Cup.', named_entities=['Sunday:::DATE', 'Sunday:::DATE', 'afternoon:::TIME', 'Eastern Standard Time:::LOC', 'Colin Montgomerie:::PERSON', 'Americans:::NORP', 'the Ryder Cup:::EVENT']),
 Row(category='Sci/Tech', text=' A swarm of earthquakes #151; one a magnitude 5.5 #151; jolted a remote, sparsely populated area along the eastern Sierra Nevada on Saturday, authorities said.', named_entities=['151:::MONEY', 'one:::CARDINAL', '5.5:::CARDINAL', '151:::MONEY', 'Sierra Nevada:::ORG', 'Saturday:::DATE']),
 Row(category='World', text=' U.S. officials have arrested two men whom they accuse of funneling money to the Palestinian militant group Hamas for the past 15 years, the Justice Department said on Friday.', named_entities=['U.S.:::GPE', 'two:::CARDINAL', 'Palestinian:::NORP', 'Hamas:::ORG', 'the past 15 years:::DATE', 'the Justice Department:::ORG', 'Friday:::DATE']),
 Row(category='World', text='By John Deane, Chief Political Correspondent, PA News, in Darfur. Foreign Secretary Jack Straw arrived in the violence-plagued Darfur region of Sudan today to experience at first hand the plight of refugees driven from their homes by marauding militiamen.', named_entities=['John Deane:::PERSON', 'PA News:::ORG', 'Darfur:::GPE', 'Jack Straw:::PERSON', 'Darfur:::GPE', 'Sudan:::GPE', 'today:::DATE', 'first:::ORDINAL']),
 Row(category='Business', text='MIAMI The nation #39;s top hurricane forecaster says this year #39;s deadly storm season was a case of the nation #39;s luck running out -- adding that future years will likely bring more frequent and more powerful storms.', named_entities=['MIAMI:::ORG', '39;s:::MONEY', '39;s:::MONEY', 'storm season:::DATE', '39;s:::MONEY', 'future years:::DATE']),
 Row(category='Sports', text='Arsenal made an ignominious piece of history yesterday when Arsne Wenger was fined 15,000 - a record for a manager - and severely reprimanded by the Football Association ', named_entities=['yesterday:::DATE', 'Wenger:::PERSON', '15,000:::CARDINAL', 'the Football Association:::ORG']),
 Row(category='Sci/Tech', text='photos The NowHouse project in San Francisco brings together the latest in design, construction and digital wares."\n', named_entities=['NowHouse:::ORG', 'San Francisco:::GPE']),
 Row(category='Sports', text=' Six time Olympic champion Michael Phelps was arrested last week and charged with drunken driving.', named_entities=['Six:::CARDINAL', 'Michael Phelps:::PERSON', 'last week:::DATE']),
 Row(category='World', text='MIAMI, Oct 21 (AFP) - Known as the capital of Cuban exiles, Miami was abuzz Thursday with news of President Fidel Castro #39;s accident, with many Cuban-Americans bemoaning that the communist leader only fell on his face, and not from power.', named_entities=['MIAMI:::ORG', 'Oct 21:::DATE', 'Cuban:::NORP', 'Miami:::GPE', 'Thursday:::DATE', 'Fidel Castro:::PERSON', '39;s:::MONEY', 'Cuban:::NORP', 'communist:::NORP']),
 Row(category='World', text=' Substantial threats to the integrity of the U.S. presidential election remain despite an improvement in election practices since 2000, an international delegation

Get embeddings

In [ ]:
%%time

# Apply the UDF to your Spark DataFrame
result = spark_df.withColumn("embeddings", embeddings_with_spacy(spark_df["text"]))


result.select("embeddings").collect() 

CPU times: user 18.2 s, sys: 15 s, total: 33.2 s
Wall time: 29min 27s
Out[19]: [Row(embeddings=[-0.45488739013671875, -0.7215296626091003, -0.057778291404247284, 0.48967742919921875, 0.6850609183311462, -0.23284205794334412, 0.23470264673233032, 0.5484534502029419, -0.006904464215040207, -0.23078426718711853, -0.09991118311882019, 0.7459468245506287, 0.4924508035182953, -0.5467090010643005, 0.7345584630966187, 0.17978602647781372, -0.01988767646253109, -0.2148844301700592, 0.7463606595993042, -0.04108617082238197, 0.8376721143722534, 0.12275837361812592, 0.2523577809333801, -0.0004883799119852483, -0.3444260060787201, -0.8058571219444275, -0.23242367804050446, -0.20006974041461945, -0.47700873017311096, 0.5768774747848511, -0.2848414182662964, -0.3994550406932831, -0.031560298055410385, 0.3916619122028351, -0.6224077939987183, 0.32743608951568604, -0.021661633625626564, 0.28029122948646545, 0.28209683299064636, 0.7374581098556519, -0.04821154847741127, -0.3448766767978668, -0.1570625752210617, -0.5937341451644897, -0.07286462932825089, 0.15723347663879395, -0.0389513336122036, -0.34276145696640015, -0.07675445079803467, -0.023630429059267044, -0.2841876447200775, 0.3803683817386627, 0.16086061298847198, -0.6089687347412109, 0.163237527012825, -0.47931647300720215, -0.5763202905654907, 0.0602579340338707, -0.15523630380630493, -0.6833606958389282, 0.1686309576034546, 0.5175464749336243, -0.05338108167052269, -0.532970666885376, -0.3374192714691162, -0.6588776707649231, 0.09263120591640472, 0.28616926074028015, 0.42485710978507996, -0.5312303900718689, -0.48029467463493347, -0.09128199517726898, 0.14500634372234344, 0.06025014445185661, -0.5931077599525452, -0.42357543110847473, -0.004016839899122715, -0.5942488312721252, -0.15757805109024048, -0.005645982455462217, -0.70455002784729, 0.6646553874015808, 0.013191303238272667, -0.8567412495613098, -0.6256577968597412, -0.555451512336731, -0.47624775767326355, 0.5480063557624817, 0.31405845284461975, -0.0498553141951561, 0.22126486897468567, 0.5722939968109131, -0.3149909973144531, 0.4837583005428314, 0.6218404173851013, 0.20621800422668457, 0.837356448173523, 0.3365787863731384, 0.3612975776195526, 0.06553412973880768, 0.13824449479579926, 0.17073360085487366, -0.3281289041042328, -0.02593475580215454, 0.2602357566356659, 0.19164438545703888, 0.024689897894859314, -0.38329190015792847, 0.4765916168689728, -0.6217994093894958, -0.06835836917161942, 0.36431685090065, -0.4836997091770172, 0.5838420987129211, -0.06094270199537277, 0.5321504473686218, -0.08759406208992004, -0.19686363637447357, -0.3980761170387268, 0.2828287184238434, 0.37406325340270996, -0.3337353467941284, -0.0009956849971786141, -0.33590060472488403, -0.3173357844352722, -0.6368738412857056, -0.2788829207420349, -0.20392416417598724, 0.41733136773109436, 0.25758063793182373, -0.2298748642206192, -0.3023558259010315, 0.32086196541786194, -0.16625669598579407, -0.63898766040802, 0.13839910924434662, -0.22220052778720856, 0.4580463469028473, 0.5313794016838074, -0.0571974441409111, -0.5459575057029724, -0.3542034327983856, -0.07917734235525131, -0.4891708195209503, 0.08875169605016708, -0.6196891665458679, 0.15434958040714264, 0.35630908608436584, -0.025103410705924034, -0.6795825958251953, -0.30334922671318054, -0.03497540205717087, -0.23688830435276031, 0.6698927283287048, 0.4324120581150055, -0.43287867307662964, 0.3237191140651703, 0.3264228105545044, 0.06998889893293381, -0.03882764279842377, -0.5740835070610046, 0.5190730690956116, -0.2199058085680008, 0.020713182166218758, 0.5447643995285034, 0.502403974533081, 0.6165183782577515, -0.4564390182495117, -0.672731339931488, -0.5248367786407471, 0.39637914299964905, 0.2420814484357834, 0.11897356808185577, 0.2662447988986969, 0.257273405790329, -0.01475232932716608, -0.48137083649635315, -0.6190000176429749, 0.2508440911769867, 0.7340704798698425, -0.17961092293262482, 0.13560137152671814, 0.3970329463481903, 0.29537278413772583, -0.16091260313987732, -0.116655565

# Spacy with Arrow Disabled

clear all variables, and rerun initialize spark, get data and define udf function sections 

In [ ]:
spark.conf.get("spark.sql.execution.arrow.pyspark.enabled")

Out[16]: 'false'

In [ ]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")

In [ ]:
spark.conf.get("spark.sql.execution.arrow.pyspark.enabled")

Out[18]: 'false'

NER extraction

In [ ]:
%%time

# Apply the UDF to your Spark DataFrame
result = spark_df.withColumn("named_entities", ner_with_spacy(spark_df["text"]))


result.collect() 

CPU times: user 4.35 s, sys: 1.11 s, total: 5.46 s
Wall time: 5min 4s
Out[20]: [Row(category='World', text='MIAMI, Oct 21 (AFP) - Known as the capital of Cuban exiles, Miami was abuzz Thursday with news of President Fidel Castro #39;s accident, with many Cuban-Americans bemoaning that the communist leader only fell on his face, and not from power.', named_entities=['MIAMI:::ORG', 'Oct 21:::DATE', 'Cuban:::NORP', 'Miami:::GPE', 'Thursday:::DATE', 'Fidel Castro:::PERSON', '39;s:::MONEY', 'Cuban:::NORP', 'communist:::NORP']),
 Row(category='World', text=' Substantial threats to the integrity of the U.S. presidential election remain despite an improvement in election practices since 2000, an international delegation of election observers reported on Thursday.', named_entities=['U.S.:::GPE', '2000:::DATE', 'Thursday:::DATE']),
 Row(category='Business', text='MIAMI The nation #39;s top hurricane forecaster says this year #39;s deadly storm season was a case of the nation #39;s luck running out -- adding that future years will likely bring more frequent and more powerful storms.', named_entities=['MIAMI:::ORG', '39;s:::MONEY', '39;s:::MONEY', 'storm season:::DATE', '39;s:::MONEY', 'future years:::DATE']),
 Row(category='Sports', text='Arsenal made an ignominious piece of history yesterday when Arsne Wenger was fined 15,000 - a record for a manager - and severely reprimanded by the Football Association ', named_entities=['yesterday:::DATE', 'Wenger:::PERSON', '15,000:::CARDINAL', 'the Football Association:::ORG']),
 Row(category='Sci/Tech', text='Servers for small businesses are one of the fastest-growing areas of the IT market, according to market researcher IDC. Much of this is down to the Small Business Server (SBS) software ', named_entities=['IDC:::ORG', 'the Small Business Server:::ORG']),
 Row(category='World', text=' The U.N. Security Council was set Friday to adopt a Russian drafted resolution that seeks to expand the prosecution and extradition of terrorist groups and individuals, such as Chechen separatist leaders.', named_entities=['The U.N. Security Council:::ORG', 'Friday:::DATE', 'Russian:::NORP', 'Chechen:::NORP']),
 Row(category='Sci/Tech', text='photos The NowHouse project in San Francisco brings together the latest in design, construction and digital wares."\n', named_entities=['NowHouse:::ORG', 'San Francisco:::GPE']),
 Row(category='Sports', text=' Six time Olympic champion Michael Phelps was arrested last week and charged with drunken driving.', named_entities=['Six:::CARDINAL', 'Michael Phelps:::PERSON', 'last week:::DATE']),
 Row(category='World', text=' U.S. officials have arrested two men whom they accuse of funneling money to the Palestinian militant group Hamas for the past 15 years, the Justice Department said on Friday.', named_entities=['U.S.:::GPE', 'two:::CARDINAL', 'Palestinian:::NORP', 'Hamas:::ORG', 'the past 15 years:::DATE', 'the Justice Department:::ORG', 'Friday:::DATE']),
 Row(category='World', text='By John Deane, Chief Political Correspondent, PA News, in Darfur. Foreign Secretary Jack Straw arrived in the violence-plagued Darfur region of Sudan today to experience at first hand the plight of refugees driven from their homes by marauding militiamen.', named_entities=['John Deane:::PERSON', 'PA News:::ORG', 'Darfur:::GPE', 'Jack Straw:::PERSON', 'Darfur:::GPE', 'Sudan:::GPE', 'today:::DATE', 'first:::ORDINAL']),
 Row(category='Sports', text='This past Sunday night (or Sunday afternoon Eastern Standard Time), my roommates and I hung our heads in defeat as Colin Montgomerie put away the Americans in the Ryder Cup.', named_entities=['Sunday:::DATE', 'Sunday:::DATE', 'afternoon:::TIME', 'Eastern Standard Time:::LOC', 'Colin Montgomerie:::PERSON', 'Americans:::NORP', 'the Ryder Cup:::EVENT']),
 Row(category='Sci/Tech', text=' A swarm of earthquakes #151; one a magnitude 5.5 #151; jolted a remote, sparsely populated area along the eastern Sierra Nevada on Saturday, authorities said.', named_entities

Get embeddings

In [ ]:
%%time

# Apply the UDF to your Spark DataFrame
result = spark_df.withColumn("embeddings", embeddings_with_spacy(spark_df["text"]))


result.collect() 

CPU times: user 19 s, sys: 16.4 s, total: 35.4 s
Wall time: 29min 30s
Out[21]: [Row(category='Sci/Tech', text='Servers for small businesses are one of the fastest-growing areas of the IT market, according to market researcher IDC. Much of this is down to the Small Business Server (SBS) software ', embeddings=[-0.2302524745464325, -0.7194628715515137, -0.1498488336801529, 0.5372987389564514, 0.6543366312980652, -0.2512519359588623, 0.16364441812038422, 0.49963563680648804, 0.037973176687955856, -0.22324985265731812, -0.16327941417694092, 0.724642276763916, 0.5325960516929626, -0.6208184361457825, 0.7467226982116699, 0.300487756729126, 0.11776601523160934, -0.05090935900807381, 0.7777619361877441, 0.0871998742222786, 0.8456060290336609, 0.15376096963882446, 0.25001633167266846, 0.016202345490455627, -0.41188961267471313, -0.7872547507286072, -0.2814028263092041, -0.1680499166250229, -0.3567851185798645, 0.5706916451454163, -0.33021309971809387, -0.30841580033302307, -0.021478503942489624, 0.3863130807876587, -0.6028757095336914, 0.4224662780761719, -0.07939791679382324, 0.15476670861244202, 0.3313465118408203, 0.6812631487846375, -0.024670418351888657, -0.3848167955875397, -0.26707395911216736, -0.5388939380645752, -0.06261628121137619, 0.08199924230575562, -0.08033236861228943, -0.46192729473114014, 0.011999143287539482, -0.04078676179051399, -0.25673750042915344, 0.3655778765678406, 0.10822781920433044, -0.6155035495758057, 0.08557606488466263, -0.4271603524684906, -0.5274438858032227, 0.0310526005923748, -0.23414018750190735, -0.7416709661483765, 0.06449643522500992, 0.5882524847984314, -0.1213822215795517, -0.523664653301239, -0.3324131369590759, -0.6452367305755615, 0.009101791307330132, 0.25643399357795715, 0.32761695981025696, -0.5405172109603882, -0.4161393642425537, 0.1087200716137886, 0.06758790463209152, 0.1856226921081543, -0.5915176272392273, -0.3922072649002075, -0.07282613962888718, -0.5860261917114258, -0.06571979820728302, -0.07433860749006271, -0.66370689868927, 0.6805481314659119, -0.04254407435655594, -0.8788973093032837, -0.6131494641304016, -0.5077494978904724, -0.46570876240730286, 0.6199180483818054, 0.3619012236595154, -0.008733248338103294, 0.20663996040821075, 0.5213069319725037, -0.3170050084590912, 0.4213531017303467, 0.6195665597915649, 0.22411300241947174, 0.809499204158783, 0.33597755432128906, 0.39851927757263184, 0.018873827531933784, 0.22909489274024963, 0.10724277049303055, -0.19772760570049286, 0.10935276001691818, 0.36698588728904724, 0.27003130316734314, 0.2159843146800995, -0.3674979507923126, 0.43382352590560913, -0.633183479309082, 0.021350696682929993, 0.3137138783931732, -0.45556819438934326, 0.5455601811408997, 0.060053158551454544, 0.587794303894043, -0.1885773092508316, -0.2400410771369934, -0.4950963854789734, 0.16895189881324768, 0.3190884590148926, -0.25919800996780396, -0.057117294520139694, -0.3203439712524414, -0.2683873176574707, -0.5502898097038269, -0.3266681730747223, -0.1071215495467186, 0.3282911777496338, 0.33713871240615845, -0.2691936492919922, -0.2962479293346405, 0.3453974723815918, -0.08041570335626602, -0.5655733346939087, 0.2082344889640808, -0.2639429271221161, 0.4194418489933014, 0.48996028304100037, 0.1216728687286377, -0.49572786688804626, -0.3782746493816376, -0.14012791216373444, -0.5493077635765076, 0.07769324630498886, -0.6439915299415588, 0.21802477538585663, 0.2501915991306305, -0.039600491523742676, -0.6364129185676575, -0.27815285325050354, -0.10734419524669647, -0.330789715051651, 0.62617427110672, 0.45911216735839844, -0.37517914175987244, 0.2881721556186676, 0.39236724376678467, -0.05044763535261154, -0.10094304382801056, -0.5070444345474243, 0.497907429933548, -0.2756999135017395, 0.05166349560022354, 0.4499090015888214, 0.42221584916114807, 0.5399430394172668, -0.49641484022140503, -0.7472755312919617, -0.4813748598098755, 0.21049664914608002, 0.27124080061912537, 0.17362207174301147, 0.08235204964876175, 0.27846530079841614, -0.20576824247837067